# Second cleaning Script 

* Contract flipped with department
* Salaries into numerical

In [330]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [331]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999


import time
import unicodedata

import datetime
from datetime import timedelta, datetime

import csv
import os

import sys
sys.path.append('./lib/')
from functions import *
from tqdm import tnrange, tqdm_notebook
from unidecode import unidecode

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import glob

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style("darkgrid")

import locale
locale.setlocale(locale.LC_ALL, 'es_ES')

'es_ES'

# Import data

In [332]:
period = '20180210-3'

In [333]:
df = pd.read_csv('../data/output/01_scraped_data_clean_first_{}.csv'.format(period), encoding='utf-8',
                 low_memory=False, sep='|', quoting=1)

In [334]:
df.shape

(1943782, 23)

# Null names?

In [335]:
df.loc[pd.isnull(df['givenn'])]

,entity,department,contract,year,month,group,lastn,lastn2,givenn,degree,qualif,position,region,specials,currency,salary,overt,start_date,end_date,obs,url,other,filename


# Shifted table because of extra column 

## Gendarmeria

In [336]:
scope = ((df['lastn'].str.contains('profesio')) | (df['lastn'].str.contains('administr'))) & \
                    (df['contract'].str.contains('Gendarme'))
df.loc[scope].sample(1)

,entity,department,contract,year,month,group,lastn,lastn2,givenn,degree,qualif,position,region,specials,currency,salary,overt,start_date,end_date,obs,url,other,filename
1174251,Ministerio de Justicia y Derechos Humanos,per_contrata,Gendarmería de Chile,Agosto,allyear,CONTRATA,profesionales,cazenave,sandoval,LORETO ANDREA,6,INGENIERO COMERCIAL,ENCARGADO ADMINISTRATIVO UNIDAD,RM,9,PESOS,1.661.211,1.338.161,NO,21/03/2011,31/12/2011,12,../data/output/scraped_data_20180210-3.csv


In [337]:
df.loc[scope, 'month'], df.loc[scope, 'year'] = df.loc[scope, 'year'], df.loc[scope, 'month']


col_shift = list(df.columns[5:-5])
for i,j in zip(col_shift, col_shift[1:]):
    df.loc[scope, i] = df.loc[scope, j].copy()

col_shift = list(['start_date', 'end_date', 'obs', 'url'])
for i,j in zip(col_shift, col_shift[2:]):
    df.loc[scope, i] = df.loc[scope, j].copy()
    #print(i,j)

df.loc[scope, ['obs', 'url']] = None

## Vialidad 

In [338]:
scope = ((df['lastn'].str.contains('profesio')) | (df['lastn'].str.contains('administr'))) & \
                    (df['contract'].str.contains('Vialidad')) & (df['group'] == '-1')

In [339]:
#df.loc[scope].sample(5)

In [340]:
col_shift = list(df.columns[6:10])
for i,j in zip(col_shift, col_shift[1:]):
    df.loc[scope, i] = df.loc[scope, j].copy()
    #print(i,j)

col_shift = list(df.columns[10:-1])
for i,j in zip(col_shift, col_shift[1:]):
    df.loc[scope, i] = df.loc[scope, j].copy()
    #print(i,j)

df.loc[scope, ['degree', 'other']] = None

## What's left

In [341]:
scope = ((df['lastn'].str.contains('profesio')) | df['lastn'].str.contains('administra'))
df.loc[scope, 'contract'].value_counts(dropna=False) / df.shape[0]

Servicio de Registro Civil e Identificación (SRCEI)    0.003446
Dirección de Vialidad                                  0.003276
Servicio de Impuestos Internos (SII)                   0.002737
Instituto de Previsión Social (IPS)                    0.002075
Consejo de Defensa del Estado                          0.000588
Dirección de Obras Portuarias (DOP)                    0.000380
Subsecretaría de Relaciones Exteriores                 0.000192
SERVIU V Región                                        0.000179
Servicio Nacional del Adulto Mayor (SENAMA)            0.000146
Instituto Nacional de la Juventud (INJUV)              0.000099
Dotación de planta                                     0.000073
Gobernación de Los Andes                               0.000062
SERVIU VII Región                                      0.000039
Dotación a contrata                                    0.000031
Name: contract, dtype: float64

# Contract filpped with Department 

In [342]:
df.loc[df['department'].str.contains('per_')].head(5)

,entity,department,contract,year,month,group,lastn,lastn2,givenn,degree,qualif,position,region,specials,currency,salary,overt,start_date,end_date,obs,url,other,filename
174015,Ministerio del Interior y Seguridad Pública,per_contrata,Gobernación de Los Andes,2017,allyear,1,profesional,aedo,torres,BORIS CRISTIAN,,ADMINISTRACION PUBLICA,PROFESIONAL,5,1,Pesos,975.647,NO,15/01/2017,31/12/2017,PAGO PROPORCIONAL A LOS DÍAS TRABAJADOS EN EL...,http://www.interior.gov.cl/transparenciaactiva...,../data/output/scraped_data_20180210-3.csv
174016,Ministerio del Interior y Seguridad Pública,per_contrata,Gobernación de Los Andes,2017,allyear,2,profesional,cubillos,cubillos,NICOLAS ANDRES,,INGENIERO EN REVENCION DE RIESGO Y MEDIO AMBI...,COORDINADOR DELEGADO DEL PASO FRONTERIZO LOS ...,5,1,Pesos,1.829.338,NO,01/01/2017,31/12/2017,,http://www.interior.gov.cl/transparenciaactiva...,../data/output/scraped_data_20180210-3.csv
174017,Ministerio del Interior y Seguridad Pública,per_contrata,Gobernación de Los Andes,2017,allyear,3,profesional,inostroza,arancibia,IVAN ENRIQUE,,ABOGADO,ASESOR JURIDICO DE LA GOBERNACIÓN PROVINCIAL ...,5,1,Pesos,1.829.338,NO,01/01/2017,31/12/2017,,http://www.interior.gov.cl/transparenciaactiva...,../data/output/scraped_data_20180210-3.csv
174018,Ministerio del Interior y Seguridad Pública,per_contrata,Gobernación de Los Andes,2017,allyear,4,profesional,rosende,bey,VERONICA CECILIA,,TRABAJADOR SOCIAL,PROFESIONAL,5,2,Pesos,1.849.118,NO,01/01/2017,31/12/2017,,http://www.interior.gov.cl/transparenciaactiva...,../data/output/scraped_data_20180210-3.csv
174019,Ministerio del Interior y Seguridad Pública,per_contrata,Gobernación de Los Andes,2017,allyear,5,profesional,hernandez,flores,MAURICIO BRYAM,,CONTADOR;INGENIERO INFORMATICO,ENCARGADO DE ADMINISTRACION Y FINANZAS,5,1,Pesos,1.680.841,NO,01/01/2017,31/12/2017,,http://www.interior.gov.cl/transparenciaactiva...,../data/output/scraped_data_20180210-3.csv


In [343]:
flipColumns(df, 'department', 'contract', 'per_')

139,638 rows of data with flipped department and contract (7.2%)
Involved department:
Servicio de Impuestos Internos (SII)                     47381
Dirección de Vialidad                                    35588
Gendarmería de Chile                                     30270
Servicio de Registro Civil e Identificación (SRCEI)       8928
Instituto de Previsión Social (IPS)                       6872
Parque Metropolitano de Santiago                          2564
Consejo de Defensa del Estado                             1887
Dirección de Obras Portuarias (DOP)                       1030
Superintendencia de Electricidad y Combustibles (SEC)     1001
SERVIU VII Región                                         1001
Subsecretaría de Economía                                  662
SERVIU V Región                                            452
Subsecretaría de Relaciones Exteriores                     448
Dirección General de Crédito Prendario (DICREP)            407
Servicio Nacional del Adulto May

In [344]:
df.loc[df['department'].str.contains('Dotaci')]

,entity,department,contract,year,month,group,lastn,lastn2,givenn,degree,qualif,position,region,specials,currency,salary,overt,start_date,end_date,obs,url,other,filename


# Salary into int 

## Cleaning script 

In [345]:
df['salary1'] = df['salary'].replace(np.nan, '-1').map(lambda x: ''.join([i for i in x if i.isdigit()])).astype(str)

clean_vals = ['', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
for c in clean_vals:
    df.loc[df['salary1'] == c, 'salary1'] = None

contains_vals = ['/', 'http']
for c in contains_vals:
    df.loc[df['salary'].str.contains(c, na=False), 'salary1'] = None


In [346]:
df['salary1'] = pd.to_numeric(df['salary1'], errors='coerce')

## Checks 

## Results 

In [347]:
pd.isnull(df['salary1']).value_counts() / df.shape[0]

False    0.840004
True     0.159996
Name: salary1, dtype: float64

17.8% of salaries are null

In [348]:
df.loc[pd.isnull(df['salary1'])]

,entity,department,contract,year,month,group,lastn,lastn2,givenn,degree,qualif,position,region,specials,currency,salary,overt,start_date,end_date,obs,url,other,filename,salary1
916,Presidencia de la República,Presidencia de la República,Dotación de planta,Año 2009,allyear,Autoridad de Gobierno,bachelet,jeria,verónica michelle,-1,-1,Presidenta de la República,RM,-1,-1,0,-1,11/03/2006,10/01/2010,1,http://www.gobiernotransparentechile.cl/direct...,-1,../data/output/scraped_data_20180210-3.csv,NaN
917,Presidencia de la República,Presidencia de la República,Dotación de planta,Año 2009,allyear,Directivo,riquelme,urra,cristian gonzalo,-1,-1,Director Administrativo,RM,-1,-1,0,-1,11/03/2006,Indefinido,1; 2=92%,http://www.gobiernotransparentechile.cl/direct...,-1,../data/output/scraped_data_20180210-3.csv,NaN
918,Presidencia de la República,Presidencia de la República,Dotación de planta,Año 2009,allyear,Directivo,vega,sanchez,francisco gabriel,-1,-1,Jefe Departamento de Finanzas,RM,-1,-1,0,-1,09/05/2006,Indefinido,1; 2=47%,http://www.gobiernotransparentechile.cl/direct...,-1,../data/output/scraped_data_20180210-3.csv,NaN
919,Presidencia de la República,Presidencia de la República,Dotación de planta,Año 2009,allyear,Directivo,vial,reyes,cecilia berta,-1,-1,Subdirectora Administrativa,RM,-1,-1,0,-1,11/03/2006,Indefinido,1; 2=91%,http://www.gobiernotransparentechile.cl/direct...,-1,../data/output/scraped_data_20180210-3.csv,NaN
920,Presidencia de la República,Presidencia de la República,Dotación de planta,Año 2009,allyear,Directivo,bustos,lynch,patricio julio,-1,-1,Jefe Departamento de Servicios Generales,RM,-1,-1,0,-1,01/07/1996,Indefinido,-1,http://www.gobiernotransparentechile.cl/direct...,-1,../data/output/scraped_data_20180210-3.csv,NaN
921,Presidencia de la República,Presidencia de la República,Dotación de planta,Año 2009,allyear,Directivo,fernández,arroyo,julio césar,-1,-1,Jefe Departamento de Abastecimiento,RM,-1,-1,0,-1,01/10/1992,Indefinido,-1,http://www.gobiernotransparentechile.cl/direct...,-1,../data/output/scraped_data_20180210-3.csv,NaN
922,Presidencia de la República,Presidencia de la República,Dotación de planta,Año 2009,allyear,Directivo,ozimica,parada,verónica del carmen,-1,-1,Jefa Departamento de Contabilidad,RM,-1,-1,0,-1,01/09/1997,Indefinido,2 =71%,http://www.gobiernotransparentechile.cl/direct...,-1,../data/output/scraped_data_20180210-3.csv,NaN
923,Presidencia de la República,Presidencia de la República,Dotación de planta,Año 2009,allyear,Directivo,tobar,marchant,fernando alfonso,-1,-1,Jefe Departamento de Movilización,RM,-1,-1,0,-1,01/02/2008,Indefinido,-1,http://www.gobiernotransparentechile.cl/direct...,-1,../data/output/scraped_data_20180210-3.csv,NaN
924,Presidencia de la República,Presidencia de la República,Dotación de planta,Año 2009,allyear,Directivo,izquierdo,silva,claudia teresa,-1,-1,"Asistente Administrativa, Dirección de Polític...",RM,-1,-1,0,-1,01/01/2007,Indefinido,-1,http://www.gobiernotransparentechile.cl/direct...,-1,../data/output/scraped_data_20180210-3.csv,NaN
925,Presidencia de la República,Presidencia de la República,Dotación de planta,Año 2009,allyear,Directivo,gonzález,castillo,adriana de lourdes,-1,-1,Contadora de Servicio de Bienestar,RM,-1,-1,0,-1,01/01/2007,Indefinido,-1,http://www.gobiernotransparentechile.cl/direct...,-1,../data/output/scraped_data_20180210-3.csv,NaN


In [349]:
df.loc[pd.isnull(df['salary1']) & (pd.notnull(df['entity'])), 'year'].value_counts()

Año 2009                        52339
Año 2008                        48799
Año 2007                        45169
Año 2006                        39976
Año 2010                        18207
2009                            13056
2010                            10318
Año 2016                         9010
2013                             8872
Año 2012                         8641
Año 2013                         6364
Año 2015                         6217
Año 2014                         5644
Año 2017                         4986
Año 2011                         4199
A�o 2009                         2654
A�o 2010                         1917
Ver Histórico                    1303
Histórico                        1272
Año 2009 (Abril-Diciembre)       1262
A�o 2006                         1141
A�o 2008                         1070
A�o 2007                         1063
2012                             1043
allyear                           935
A�o 2015                          795
Diciembre   

# Print output 

In [351]:
df.to_csv('../data/output/02_scraped_data_clean_second_{}.csv'.format(period), encoding='utf-8',index=False,
          sep='|', quoting=1)

In [352]:
df.shape

(1943782, 24)